# Taxi Stay-Point Detection with RAPIDS (cuDF & cuPy)

This notebook implements a highly efficient, GPU-accelerated workflow to identify stay-points from taxi GPS data.

**The overall logic is as follows:**

1.  **Pre-filter in Database**: Extract only relevant low-velocity GPS points from the database to minimize data transfer and memory footprint.
2.  **Load Data to GPU**: Read the pre-filtered data into a cuDF DataFrame.
3.  **Identify Segments**: Detect consecutive sequences of low-velocity points by checking the time (`Δt`) and distance (`Δd`) gaps between them. A large gap indicates a new stay segment.
4.  **Aggregate Segments**: Group the points by segment and calculate metrics for each potential stay-point (start/end times, duration, centroid location).
5.  **Filter by Duration**: Keep only the segments that meet the minimum stay duration threshold (`t_min`).
6.  **Persist Results**: Save the final stay-points to a file or database for further analysis.



## Step 0: Parameters & Environment Setup


In [1]:
import cudf
import cupy as cp
from sqlalchemy import create_engine
import sqlalchemy # For specifying dtype in to_sql
import pandas as pd
import os
from urllib.parse import quote_plus
import subprocess
import psycopg2
import numpy as np

# parmaeters

In [24]:

# Thresholds for stay-point detection
VELOCITY_THRESHOLD_KMH = 1.0   # (v_max) Max speed to be considered "low-velocity"
TIME_GAP_THRESHOLD_S =120      # (gap_thr/s) Max time gap in seconds between consecutive points in the same stay
DISTANCE_GAP_THRESHOLD_M = 100  # (dist_thr/m) Optional: Max distance between points to break a stay segment
MIN_STAY_DURATION_S = 600       # (t_min/s) Minimum duration for a sequence of points to be considered a valid stay


# Database connection (optional, for writing results)
username = 'xuhang.liu'
password = 'xuhangLIU@HOMES'
hostname = 'homes-database.epfl.ch'
port = '30767'
dbname = 'Shenzhen_Taxi'
password = quote_plus(password)
DB_URL = f"postgresql://{username}:{password}@{hostname}:{port}/{dbname}"


In [14]:
# --- Custom GPU Haversine Distance Function (replaces cuspatial) ---
def haversine_distance_gpu(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in meters between two points 
    on the earth (specified in decimal degrees) using cupy for GPU acceleration.
    """
    # cupy operations require float64 for precision in trig functions
    lon1_rad = cp.radians(lon1.astype(cp.float64))
    lat1_rad = cp.radians(lat1.astype(cp.float64))
    lon2_rad = cp.radians(lon2.astype(cp.float64))
    lat2_rad = cp.radians(lat2.astype(cp.float64))

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = cp.sin(dlat / 2)**2 + cp.cos(lat1_rad) * cp.cos(lat2_rad) * cp.sin(dlon / 2)**2
    c = 2 * cp.arcsin(cp.sqrt(a))
    
    # Radius of earth in meters.
    R = 6371000
    return c * R



## Step 1: Data Loading & Preprocessing

In [3]:
table_name = "taxi_data2020_01_01"
OUTPUT_CSV_PATH = os.path.join(os.getcwd(), 'gps_data_01_01.csv')
print(f"当前目录: {os.getcwd()}")
print(f"输出文件: {OUTPUT_CSV_PATH}")

当前目录: /home/lxhep/etaxi
输出文件: /home/lxhep/etaxi/gps_data_01_01.csv


In [7]:

query = f"""
SELECT
    gid, taxiid,
    to_char(time, 'YYYY-MM-DD HH24:MI:SS.US') as time,
    lon, lat, velocity, angle, passenger, zoneid, validity,
    hvalue, "OSM_edgeid", "OSM_distance"
FROM {table_name}
WHERE  passenger = 0 AND velocity =0 AND time >= '2020-01-01'
ORDER BY taxiid, time
"""

conn = psycopg2.connect(DB_URL)
cur = conn.cursor()

with open(OUTPUT_CSV_PATH, 'w', encoding='utf-8') as f:
    cur.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", f)

cur.close()
conn.close()

In [6]:

INPUT_CSV_PATH = 'gps_data_01_01.csv'

gdf_01_01= cudf.read_csv(
        INPUT_CSV_PATH,
        nrows=1E6,
        dtype={
            'taxiid': 'str',
            'lon': np.float32,
            'lat': np.float32,
            'angle': np.int16,
            'zoneid': np.int32,
            'validity': np.int8
        },
        parse_dates=['time']
    )



In [33]:
gdf_01_01.iloc[1000]

,gid,taxiid,time,lon,lat,velocity,angle,passenger,zoneid,validity,hvalue,OSM_edgeid,OSM_distance,prev_time,prev_lon,prev_lat,dt,dd,grp
1000,79995452,UUUB0C0M7,2020-01-03 06:46:52,114.008827,22.539022,0,45,0,1863,1,H,"{380820589,493389662}",6.527811,2020-01-03 06:46:50,114.008827,22.539022,2,0.0,287


In [12]:
gdf_01_01 = gdf_01_01.dropna()

# timegap


In [23]:
if not gdf_01_01.empty:
    # --- Calculate gaps within each taxi's trajectory ---

    # 1. Get the previous point's time and coordinates
    # groupby().shift() is a powerful operation to look at the previous record within a group
    gdf_01_01['prev_time'] = gdf_01_01.groupby('taxiid')['time'].shift()
    gdf_01_01['prev_lon'] = gdf_01_01.groupby('taxiid')['lon'].shift().fillna(0)
    gdf_01_01['prev_lat'] = gdf_01_01.groupby('taxiid')['lat'].shift().fillna(0)

    # 2. Calculate time gap in seconds
    # .fillna() is used for the very first point of each taxi, which has no 'previous' time
    gdf_01_01['dt'] = (gdf_01_01['time'] - gdf_01_01['prev_time']).dt.seconds.fillna(99999)

    # 3. Calculate distance gap in meters using our custom GPU function
    # Haversine distance is the shortest distance between two points on a sphere
    gdf_01_01['dd'] = haversine_distance_gpu(
       gdf_01_01['lon'], gdf_01_01['lat'], gdf_01_01['prev_lon'], gdf_01_01['prev_lat']
    )

    print(gdf_01_01.head(10))


       gid     taxiid                time         lon        lat  velocity  \
0  3174547  UUUB0C0M7 2020-01-01 00:15:06  114.121605  22.568609         0   
1  4055440  UUUB0C0M7 2020-01-01 00:19:12  114.125244  22.566677         0   
2  3183345  UUUB0C0M7 2020-01-01 00:19:43  114.125198  22.567152         0   
3  3182961  UUUB0C0M7 2020-01-01 00:19:43  114.125198  22.567152         0   
4  3190190  UUUB0C0M7 2020-01-01 00:19:58  114.125198  22.567152         0   
5  3972098  UUUB0C0M7 2020-01-01 00:20:13  114.125198  22.567152         0   
6  4889514  UUUB0C0M7 2020-01-01 00:20:28  114.125198  22.567152         0   
7  3978957  UUUB0C0M7 2020-01-01 00:20:28  114.125198  22.567152         0   
8  3979415  UUUB0C0M7 2020-01-01 00:20:30  114.125198  22.567152         0   
9  5900661  UUUB0C0M7 2020-01-01 00:33:29  114.163345  22.566698         0   

   angle  passenger  zoneid  validity hvalue   OSM_edgeid  OSM_distance  \
0     90          0    1205         1      H  {913520745}     47.1

## Step 3: Identify Stay Segments

A new stay segment begins when:
1. It's the first point for a given taxi.
2. The time gap (`dt`) from the previous point exceeds `TIME_GAP_THRESHOLD_S`.
3. The distance gap (`dd`) from the previous point exceeds `DISTANCE_GAP_THRESHOLD_M`.

We create a boolean flag for these conditions and then use a cumulative sum (`cumsum`) to assign a unique ID (`grp`) to each segment.


In [25]:
if not gdf_01_01.empty:
    # --- Flag the start of a new segment ---
    new_segment_flag = (gdf_01_01['dt'] > TIME_GAP_THRESHOLD_S) | \
                       (gdf_01_01['dd'] > DISTANCE_GAP_THRESHOLD_M) | \
                       (gdf_01_01['prev_time'].isnull()) # The first point for each taxi is always a new segment

    # --- Assign a unique ID to each segment ---
    # Convert the boolean Series to a cupy array for cumsum
    # This is a highly efficient way to create group identifiers
    gdf_01_01['grp'] = cp.asarray(new_segment_flag).astype(cp.int32).cumsum()

    print("Assigned segment IDs to all points.")
    print(gdf_01_01.head(10))

Assigned segment IDs to all points.
       gid     taxiid                time         lon        lat  velocity  \
0  3174547  UUUB0C0M7 2020-01-01 00:15:06  114.121605  22.568609         0   
1  4055440  UUUB0C0M7 2020-01-01 00:19:12  114.125244  22.566677         0   
2  3183345  UUUB0C0M7 2020-01-01 00:19:43  114.125198  22.567152         0   
3  3182961  UUUB0C0M7 2020-01-01 00:19:43  114.125198  22.567152         0   
4  3190190  UUUB0C0M7 2020-01-01 00:19:58  114.125198  22.567152         0   
5  3972098  UUUB0C0M7 2020-01-01 00:20:13  114.125198  22.567152         0   
6  4889514  UUUB0C0M7 2020-01-01 00:20:28  114.125198  22.567152         0   
7  3978957  UUUB0C0M7 2020-01-01 00:20:28  114.125198  22.567152         0   
8  3979415  UUUB0C0M7 2020-01-01 00:20:30  114.125198  22.567152         0   
9  5900661  UUUB0C0M7 2020-01-01 00:33:29  114.163345  22.566698         0   

   angle  passenger  zoneid  validity hvalue   OSM_edgeid  OSM_distance  \
0     90          0    1205   

## Step 4: Aggregate Segments to Find Stays

Now that each point belongs to a segment (`grp`), we can group by `taxiid` and `grp` to aggregate the data. For each segment, we calculate:
- The start and end time.
- The average longitude and latitude (to find the centroid).
- The number of points in the stay.

Finally, we calculate the duration of each segment and filter out those that are shorter than `MIN_STAY_DURATION_S`.


In [28]:
if not gdf_01_01.empty:
    # --- Group by segment to create potential stays ---
    agg_stays = gdf_01_01.groupby(['taxiid', 'grp']).agg(
        start_time=('time', 'min'),
        end_time=('time', 'max'),
        stay_lon=('lon', 'mean'),
        stay_lat=('lat', 'mean'),
        points=('taxiid', 'count')
    ).reset_index()

    # --- Calculate duration and filter for valid stays ---
    agg_stays['duration_s'] = (agg_stays['end_time'] - agg_stays['start_time']).dt.seconds

    final_stays = agg_stays[agg_stays['duration_s'] >= MIN_STAY_DURATION_S].copy()
    final_stays = final_stays.sort_values(['taxiid', 'start_time'])
    print(f"Found {len(final_stays)} valid stay-points.")
    print("Final stay-points DataFrame head:")
    print(final_stays.head(20))


Found 6655 valid stay-points.
Final stay-points DataFrame head:
           taxiid  grp          start_time            end_time    stay_lon  \
112212  UUUB0C0M7  158 2020-01-02 03:23:09 2020-01-02 03:56:24  114.078497   
307     UUUB0C0M7  243 2020-01-02 20:03:06 2020-01-02 20:17:51  114.113434   
226268  UUUB0C0M7  258 2020-01-02 23:53:55 2020-01-03 00:08:10  114.032786   
267301  UUUB0C0M7  262 2020-01-03 00:44:40 2020-01-03 01:14:24  114.124169   
58853   UUUB0C0M7  264 2020-01-03 01:24:39 2020-01-03 01:46:40  114.124214   
142251  UUUB0C0M7  272 2020-01-03 02:39:55 2020-01-03 02:50:40  114.078691   
262662  UUUB0C0M7  278 2020-01-03 03:29:39 2020-01-03 03:43:25  114.124285   
152341  UUUB0C0M7  279 2020-01-03 03:46:10 2020-01-03 04:04:39  114.124304   
188232  UUUB0C0M7  380 2020-01-04 03:15:08 2020-01-04 03:30:21  113.810077   
228868  UUUB0C0M7  386 2020-01-04 04:34:23 2020-01-04 04:57:53  114.143025   
190714  UUUB0C0M7  495 2020-01-05 01:32:16 2020-01-05 01:46:32  114.138767   


In [30]:
len(final_stays)

6655

# Matching with stations


In [39]:

from scipy.spatial import cKDTree


In [46]:
# --- Parameters for Matching ---
STATION_CSV_PATH = 'station_information.csv'
DISTANCE_THRESHOLD_M = 100  # Threshold in meters to consider a stay as a charging event

# --- 1. Load Station Data (on CPU) ---
# Create a dummy station file if it doesn't exist for demonstration

stations_df = pd.read_csv(STATION_CSV_PATH)
# --- 2. Build cKDTree from Station Coordinates (on CPU) ---
# This assumes both GPS and station data use the WGS-84 coordinate system.
station_coords = stations_df[['longitude', 'latitude']].to_numpy()
station_tree = cKDTree(station_coords)
print("cKDTree built successfully from station data.")



cKDTree built successfully from station data.


In [47]:
# --- 3. Query the Tree with Stay-Points ---
# Check if the 'final_stays' DataFrame from the previous step exists
if not final_stays.empty:


    # 3a. Transfer stay-point coordinates from GPU to CPU
    stay_coords = final_stays[['stay_lon', 'stay_lat']].to_numpy()

    # 3b. Perform the nearest neighbor query using the cKDTree

    distances_deg, indices = station_tree.query(stay_coords, k=1)


    # 3c. Convert distance from degrees to approximate meters.
    # This approximation (1 degree ≈ 111.32 km) is valid for city-scale analysis.
    distances_m = distances_deg * 111320

    # --- 4. Add Matching Results back to the GPU DataFrame ---

    # Transfer the numpy arrays from CPU back to GPU as new cudf Series
    final_stays['distance_to_station_m'] = distances_m

    # Map the indices from the query result back to the original station_id
    station_id_lookup = stations_df['station_id'].to_numpy()
    nearest_station_ids = station_id_lookup[indices]
    final_stays['nearest_station_id'] = cudf.Series(nearest_station_ids, index=final_stays.index)

    # --- 5. Filter for Charging Events based on the distance threshold ---

    charging_events = final_stays[final_stays['distance_to_station_m'] <= DISTANCE_THRESHOLD_M].copy()
    charging_events = charging_events.sort_values(['taxiid', 'start_time'])


else:
    print("\n'final_stays' DataFrame not found or is empty. Please run the preceding cells to generate stay-points first.")


In [49]:
len(charging_events)

543

In [50]:

charging_events.iloc[200]

,taxiid,grp,start_time,end_time,stay_lon,stay_lat,points,duration_s,distance_to_station_m,nearest_station_id
144173,UUUB1C1P5,95967,2020-01-05 08:32:27,2020-01-05 09:01:12,113.941949,22.532894,45,1725,93.619151,1467
